# FIT5196 Assessment 1
#### Student Name: NIkhil Taralkar
#### Student ID: 28980433

Date: 08/04/2017

Version: 2.0

Environment: Python 2.7.0 and Anaconda 4.3.0 (64-bit)

Libraries used: 

* Json
* Pandas
* re




## 1. Introduction
This assignment comprises the execution of pdf data processing and analysis tasks applied to documents in PDF format. This task focuses on converting the health in an PDF file ("health.pdf") into a csv file. Moreover, it mainly focuses on extracting the pdf data, rearranging the data in more redable format and converting it into csv file for further analysis.

## *Task  - Extract Data from a PDF File*

The PDF file, "health.pdf", contains the children's health data over 202 countries in the world. The table spreads over four pages. The below code is parsing pdf pages 1-3, while the 4th page is parsed using Tabula. The required task are the following:

- Correctly parse and extract the table
- It is not required to extract the column labels. Except for the first column, which should be named as "Country Name", the other columns should be indexed with integers as shown in the figure;
- If the number followed by a character "x" in the pdf file,  "x" must be dropped in your script
- Extracted table should be saved in a CSV file named as "health.csv".



## 2. Import Libraries

In [16]:
# importing Libraries

import pandas as pd
import re
from pdftables import get_tables

## 3. Reading PDF data

In [17]:
# reading pdf file

pdfFile = './health.pdf'
pdfobj = open(pdfFile, 'rb')
tables = get_tables(pdfobj)

# loop to get rid of special characters and unwanted text
for table in tables:
    for row in table:
        for i in xrange(len(row)):
            row[i] = row[i].encode('utf-8')
            row[i] = row[i].replace('\xe2\x80\x93', '-')
            row[i] = row[i].replace('\xe2\x80\x99', '\'')
        print row

['TABLE 3. H', 'EA', 'LT', 'H', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'I', 'mmuni', 'zation c', 'overage', '(%)', '', '', '', 'Pneumonia', 'Diarrhoea', '', 'Malaria', '']
['', 'Udris', 'se of bnking ervices', 'asic water  (%)', 'Use of bsanitatiservices', 'asic on  (%)', '', '', '', '', '', '', '', '', '', '', 'Protection at birth (PAB) against', 'Care seekingfor children with  symptoms of', 'Treatment with oral rehydra-tion salts', 'Care seeking for children with fever', 'Children sleep-ing under', 'House-holds with at least one']
['Countries', '', '2015', '', '2015', '', 'BCG', 'DTP1\xce\xb2', 'DTP3\xce\xb2', 'polio3', 'MCV1', 'MCV2^', 'HepB3', 'Hib3', 'rota', 'PCV3', 'tetanus\xce\xbb', 'pneumonia (%)', '(ORS) (%)', '(%)', 'ITNs (%)', 'ITN (%)']
['and areas', 'total', 'urban', 'rural', 'totalurban', 'rural', '', '', '', '', '', '2016', '', '', '', '', '', '', '2', '011-2016*', '', '']
['Afghanistan', '63', '89', '

In [18]:
first_name = ''
data1 = {}
data2 = {}

# loop to get two tables with different number of columns
for table in tables:
    for row in table[5:]:
        if row[0] == 'SUMMARY':
            break
        if row[2] == '':
            first_name = row[0]
            continue
        if first_name != '':
            row[0] = '{} {}'.format(first_name, row[0])
            first_name = ''
        if len(row) == 22:
            data1[row[0]] = row[1:]
        if len(row) == 12:
            data2[row[0]] = row[1:]
            
# Creating a dataframe for table 1
df =  pd.DataFrame(data1)
df = df.T
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Afghanistan,63,89,53,3956,33,74,73,65,60,62,...,65,65,0,65,65,62,46,63,5,26
Albania,91,93,90,9898,97,99,99,98,98,96,...,98,98,0,98,92,70x,54x,71x,-,-
Algeria,93,95,89,8790,82,99,96,91,91,94,...,91,91,0,61,92,66,25,-,-,-
Andorra,100,100,100,100100,100,-,99,98,98,97,...,94,98,0,92,-,-,-,-,-,-
Angola,41,63,23,3962,21,58,79,64,66,49,...,64,64,53,58,78,49,43,51,22,31


## 4. Arranging the data

For 3rd column, where values are joined, I am separating those values using regular expression. By checking the length of the values and accordingly putting those values to their respective places. Every value is checked for pattern in the string. Based on this pattern values are sorted. For example, some values have '-' or 'x'.

A regex is defined for separating values in the columns. The \d{2} is used for matching digits of length two, while '\d{3}' is used for matching three digits number. In some place ('-') special character is used. Therefore, it is used as  '((\d{2})(-))'


- \w Matches any alphanumeric character, which is equivalent to [a-zA-Z0-9].
- \S: Matches any non-whitespace character; which is equivalent to [^ \t\n\r\f\v].
- \d Matches any decimal digit; equivalent to the set [0-9].


In [19]:
# combining 3rd and 4th column into strings and making new column 21
df[21] = df[3].map(str) + df[4]

# spliiting column 21 into three rows
x1 = df[21].values.tolist()
xlist = []
for i in x1:
    if len(i) == 3:
        num = re.match('((-)+)', i).group(2)
        xlist.append([num, num, num])
    if len(i) == 4:
        if i.endswith('-'):
            num = re.match('((\d{2})(-))', i).group(2)
            num1 = re.match('((\d{2})(-))', i).group(3)
            xlist.append([num, num1, num1])
        else:
            num = re.match('((\d\d)(\d\d))', i).group(2)
            num1 = re.match('((\d\d)(\d\d))', i).group(3)
            xlist.append([num, num1, '-'])
    if len(i) == 5:
        
        if i.endswith('--'): 
            num = re.match('((\d{3})(-))', i).group(2)
            num1 = re.match('((\d{3})(-))', i).group(3)
            xlist.append([num, num1, num1])
        elif i.endswith('6-') or i.endswith('7-'):  
            num = re.match('((\d{2})(\d{2})(-))', i).group(2)
            num1 = re.match('((\d{2})(\d{2})(-))', i).group(3)
            num2 = re.match('((\d{2})(\d{2})(-))', i).group(4)
            xlist.append([num, num1, num2])
        elif i.endswith('-93'):
            num = re.match('((\d{2})(-)(\d{2}))', i).group(2)
            num1 = re.match('((\d{2})(-)(\d{2}))', i).group(3)
            num2 = re.match('((\d{2})(-)(\d{2}))', i).group(4)
            xlist.append([num, num1, num2])
        else:
            num = re.match('((\d{2})(\d{2})(\d))', i).group(2)
            num1 = re.match('((\d{2})(\d{2})(\d))', i).group(3)
            num2 = re.match('((\d{2})(\d{2})(\d))', i).group(4)
            xlist.append([num, num1, num2])
    
    if len(i) == 6:
        num = re.findall('\d{2}', i)
        xlist.append(num)
    
    if len(i) == 7:
        if i.endswith('-'): 
            num = re.match('((\d{3})(\d{3})(-))', i).group(2)
            num1 = re.match('((\d{3})(\d{3})(-))', i).group(4)
            xlist.append([num, num, num1])

        if i.endswith('100'):
            num = re.match('((\d{2})(\d{2})(\d{3}))', i).group(2)
            num1 = re.match('((\d{2})(\d{2})(\d{3}))', i).group(3)
            num2 = re.match('((\d{2})(\d{2})(\d{3}))', i).group(4)
            xlist.append([num, num1, num2])
        if i.endswith('99'):
            num = re.match('((\d{2})(\d{3})(\d{2}))', i).group(2)
            num1 = re.match('((\d{2})(\d{3})(\d{2}))', i).group(3)
            xlist.append([num, num1, num])
            
    if len(i) == 8:
        num = re.match('((\d{3})(\d{3})(\d{2}))', i).group(2)
        num1 = re.match('((\d{3})(\d{3})(\d{2}))',i).group(4)
        xlist.append([num, num, num1])
    
    if len(i) == 9:
        num = re.findall('\d{3}', i)
        xlist.append(num)

            
print(len(xlist))


188


In [20]:
# putting each rows into separate list
r1 = []
r2 = []
r3 = []
for i in xlist:
    r1.append(i[0])
    r2.append(i[1])
    r3.append(i[2])
    
# putting list into dataframe
df[22] = r1
df[23] = r2
df[24] = r3
df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Afghanistan,63,89,53,3956,33,74,73,65,60,62,...,65,62,46,63,5,26,395633,39,56,33
Albania,91,93,90,9898,97,99,99,98,98,96,...,92,70x,54x,71x,-,-,989897,98,98,97
Algeria,93,95,89,8790,82,99,96,91,91,94,...,92,66,25,-,-,-,879082,87,90,82
Andorra,100,100,100,100100,100,-,99,98,98,97,...,-,-,-,-,-,-,100100100,100,100,100
Angola,41,63,23,3962,21,58,79,64,66,49,...,78,49,43,51,22,31,396221,39,62,21


In [21]:
# dropping unwanted columns and rearranging the dataframe
df = df.drop([3, 4, 21], axis=1)
df = df[[0, 1, 2, 22, 23, 24, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
df.head()

,0,1,2,22,23,24,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
Afghanistan,63,89,53,39,56,33,74,73,65,60,...,65,65,0,65,65,62,46,63,5,26
Albania,91,93,90,98,98,97,99,99,98,98,...,98,98,0,98,92,70x,54x,71x,-,-
Algeria,93,95,89,87,90,82,99,96,91,91,...,91,91,0,61,92,66,25,-,-,-
Andorra,100,100,100,100,100,100,-,99,98,98,...,94,98,0,92,-,-,-,-,-,-
Angola,41,63,23,39,62,21,58,79,64,66,...,64,64,53,58,78,49,43,51,22,31


In [22]:
# renaming the columns serially
df.columns = list(range(len(df.columns)))

# replace speacial characters in the dataframe
df.replace('x', '', regex=True, inplace=True)
df.replace('-', '', regex=True, inplace=True)

df.index.name = 'Country Name' # renaming index as Country Name

df.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,63,89,53,39,56,33,74,73,65,60,...,65,65,0,65,65,62,46,63,5,26
Albania,91,93,90,98,98,97,99,99,98,98,...,98,98,0,98,92,70,54,71,,
Algeria,93,95,89,87,90,82,99,96,91,91,...,91,91,0,61,92,66,25,,,
Andorra,100,100,100,100,100,100,,99,98,98,...,94,98,0,92,,,,,,
Angola,41,63,23,39,62,21,58,79,64,66,...,64,64,53,58,78,49,43,51,22,31


## 5. Used Tabula for parsing pdf page 4.

In [31]:
# importing libraries
from tabula import read_pdf
import pandas as pd


#df1=read_pdf('health.pdf', encoding='latin-1', pages =1)
#df2=read_pdf('health.pdf',encoding = 'latin-1', pages=2)
#df3=read_pdf('health.pdf',encoding = 'latin-1', pages=3)
df4=read_pdf('health.pdf',encoding = 'latin-1', pages=4,area=[98,39,153,554]) # reading page 4 only
#top, left, top + height, left + width 

Note: I have used tabula for parsing pdf page 4, since it is very easy with tabula using the pixels measurements from pdf file.

In [32]:
# getting the required data
df1 = df4.iloc[6:]
df1.set_index(df1['Unnamed: 0'].values, inplace=True)
df1.drop(['Unnamed: 0'], 1, inplace=True)

C:\Users\nikhi\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [33]:
# using regex spliiting the columns
df1[[1, 2, 3]] = df1['Unnamed: 1'].str.split('\s*', expand=True)
df1[[4, 5, 6]] = df1['Unnamed: 2'].str.split('\s*', expand=True)
df1[[7, 8, 9, 10, 11]]=df1['Immunization coverage (%)'].str.split('[\s_]+', expand=True)


In [34]:
# removing the special characters by replacing them with space.
df1.replace(' x', '', regex=True, inplace=True)
df1.replace('â', '', regex=True, inplace=True)
        

C:\Users\nikhi\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\nikhi\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
# arranging the columns
df1[[12,13]] = df1['Pneumonia Diarrhoea'].str.split('\s*', expand=True)
df1 =df1[[1, 2, 3, 4, 5, 6,'Unnamed: 3','Unnamed: 4','Unnamed: 5', 7, 8, 9, 10, 11,'Unnamed: 7'
      ,'Unnamed: 8','Unnamed: 9',12, 13,'Unnamed: 11', 'Malaria','Unnamed: 13']]


# renaming columns serially
df1.columns = list(range(len(df1.columns)))
df1.index.name = 'Country Name'
df1.dropna(0,how ='all',inplace=True )



In [36]:
# concat two dataframes
df2 = pd.concat([df, df1])
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,63,89,53,39,56,33,74,73,65,60,...,65,65,0,65,65,62,46,63,5,26
Albania,91,93,90,98,98,97,99,99,98,98,...,98,98,0,98,92,70,54,71,,
Algeria,93,95,89,87,90,82,99,96,91,91,...,91,91,0,61,92,66,25,,,
Andorra,100,100,100,100,100,100,,99,98,98,...,94,98,0,92,,,,,,
Angola,41,63,23,39,62,21,58,79,64,66,...,64,64,53,58,78,49,43,51,22,31


In [37]:
print(df2.shape)
df2.head()

(202, 22)


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,63,89,53,39,56,33,74,73,65,60,...,65,65,0,65,65,62,46,63,5,26
Albania,91,93,90,98,98,97,99,99,98,98,...,98,98,0,98,92,70,54,71,,
Algeria,93,95,89,87,90,82,99,96,91,91,...,91,91,0,61,92,66,25,,,
Andorra,100,100,100,100,100,100,,99,98,98,...,94,98,0,92,,,,,,
Angola,41,63,23,39,62,21,58,79,64,66,...,64,64,53,58,78,49,43,51,22,31
Anguilla,98,98,,97,97,,,,,,...,,,,,,,,,,
Antigua and Barbuda,97,,,88,,,,99,99,86,...,99,99,0,0,,,,,,
Argentina,100,100,100,95,95,94,92,97,92,87,...,92,92,75,82,,94,18,,,
Armenia,99,99,99,92,96,83,99,97,94,96,...,94,94,94,94,,57,37,71,,


## 6. Converting dataframe to csv

In [38]:
# save file to csv format
df2.to_csv('health.csv', encoding='utf-8-sig')

## 7. Summary

This assessment measured the understanding of basic PDF file processing techniques in the Python programming language. The main outcomes achieved while applying these techniques were:

- **PDF parsing and data extraction**: By using the Regular expression. With functions like findall() and re..macth() and Tabula it was possible to access text data with only a few inspections.

- **Exporting data to specific format** : By using built-in functions like `DataFrame.to_csv()` it was possible to export data frames into `.csv` files without additional formatting and transformations. In other cases, native file operations like `open()` and `write()` were required where data had to be processed line by line. Encoding is required to remove and special character appearance in the output file

## 8. Referneces

- *Convert Dataframe column values to string* https://stackoverflow.com/questions/17950374/converting-a-column-within-pandas-dataframe-from-int-to-string

- *Encoding, for removing soecial characters in output file*  https://stackoverflow.com/questions/34399172/why-does-my-python-code-print-the-extra-characters-%C3%AF-when-reading-from-a-tex

- *Tabula file reading* https://github.com/chezou/tabula-py/blob/master/README.md
